**This notebook write with [tf.data.datset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) pipeline and [tf.keras](https://www.tensorflow.org/api_docs/python/tf/keras) high-level API
and base on [CS230 Stanford Tensorflow Tutorial](https://cs230.stanford.edu/blog/tensorflow/)**

*codes will be completed by the time*

**some parts of code inspired from [This](https://www.kaggle.com/gabrielmv/aerial-cactus-identification-keras) notebook**

I hope to be useful

# Import Necessary library

In [ ]:
import random
import os

import sklearn.utils
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import cv2 as cv

from tqdm import tqdm

import tensorflow as tf
#tf.enable_eager_execution()
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, BatchNormalization, Dropout, LeakyReLU, DepthwiseConv2D, Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping

import json
import logging

import numpy as np
import matplotlib.pyplot as plt 

# All Path

define path to dataset and csv file

In [ ]:
data_dir = r'../input'
dataset_dir = os.path.join(data_dir, r'train/train')
csv_dir = os.path.join(data_dir , 'train.csv')
print(os.getcwd())
print(dataset_dir)

# use for reproductibility

The most important things for compare hypreParameter is reproductibility in code
so fixed all seed at first

In [ ]:
# I set all seed as 1372

from numpy.random import seed
seed(1372)
from tensorflow import set_random_seed
set_random_seed(1372)

#df = sklearn.utils.shuffle(df,random_state=1372)

# Define functions

read ,resize and save image (due to dataset image cut to 32x32 pixel no need to that )

In [ ]:
def resize_and_save(filename, input_dir, output_dir, size=32):
    """Resize the image contained in `filename` and save it to the `output_dir`"""
    image = Image.open(os.path.join(input_dir, filename))
    # No resize Need for this dataset
    # Use bilinear interpolation instead of the default "nearest neighbor" method
    # image = image.resize((size, size), Image.BILINEAR)
    image.save(os.path.join(output_dir, filename)) # linux => / windows => \\

Base on CS230 course,It's better to save all hyperParameter to one file
after each change can compare between result and select best of them


parameter can be saved or read as **param.json** file

In [ ]:
class Params():
    """Class that loads hyperparameters from a json file.

    Example:
    ```
    params = Params(json_path)
    print(params.learning_rate)
    params.learning_rate = 0.5  # change the value of learning_rate in params
    ```
    """

    def __init__(self, json_path):
        self.update(json_path)

    def save(self, json_path):
        """Saves parameters to json file"""
        with open(json_path, 'w') as f:
            json.dump(self.__dict__, f, indent=4)

    def update(self, json_path):
        """Loads parameters from json file"""
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

    @property
    def dict(self):
        """Gives dict-like access to Params instance by `params.dict['learning_rate']`"""
        return self.__dict__


def set_logger(log_path):
    """Sets the logger to log info in terminal and file `log_path`.

    In general, it is useful to have a logger so that every output to the terminal is saved
    in a permanent file. Here we save it to `model_dir/train.log`.

    Example:
    ```
    logging.info("Starting training...")
    ```

    Args:
        log_path: (string) where to log
    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    if not logger.handlers:
        # Logging to a file
        file_handler = logging.FileHandler(log_path)
        file_handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
        logger.addHandler(file_handler)

        # Logging to console
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(logging.Formatter('%(message)s'))
        logger.addHandler(stream_handler)


def save_dict_to_json(d, json_path):
    """Saves dict of floats in json file

    Args:
        d: (dict) of float-castable values (np.float, int, float, etc.)
        json_path: (string) path to json file
    """
    with open(json_path, 'w') as f:
        # We need to convert the values to float for json (it doesn't accept np.array, np.float, )
        d = {k: float(v) for k, v in d.items()}
        json.dump(d, f, indent=4)


# Dataset Pipeline

The Dataset API allows you to build an asynchronous, highly optimized data pipeline to prevent your GPU from data starvation. It loads data from the disk (images or text), applies optimized transformations, creates batches and sends it to the GPU. Former data pipelines made the GPU wait for the CPU to load the data, leading to performance issues.

**for more info see [This](https://cs230.stanford.edu/blog/datapipeline/)**

In [ ]:
# Get the filenames in each directory (train and test)
df = pd.read_csv(csv_dir)
df['id'] = dataset_dir + '/' + df['id'].astype(str)
filenames = df['id']
labels = df['has_cactus'].astype(np.float32)


# Make sure to always shuffle with a fixed seed so that the split is reproducible
df = sklearn.utils.shuffle(df,random_state=1372)
df = df.reset_index(drop=True)

show a sample of label and filename array
operation like shuffel on filename array is faster than image array

In [ ]:
print('sample filename ',filenames[0])
print('sample label (1 = exsit) , (0 = dosent exist any cactus) ',labels[0],type(labels[0]))

## Create tf.data.dataset

**see [this](http://cs230.stanford.edu/blog/datapipeline/#building-an-image-data-pipeline) for more info**

we can map function on all image of dataset by one line
SO this can help us a lot like in agumentaion or preprocessing

In [ ]:
def _parse_function(filename, label, size):
    """Obtain the image from the filename (for both training and validation).

    The following operations are applied:
        - Decode the image from jpeg format
        - Convert to float and to range [0, 1]
    """
    image_string = tf.read_file(filename)

    # Don't use tf.image.decode_image, or the output shape will be undefined
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)

    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image_decoded, tf.float32)

    resized_image = tf.image.resize_images(image, [size, size])

    return resized_image, label

In [ ]:
def train_preprocess(image, label, use_random_flip):
    """Image preprocessing for training.

    Apply the following operations:
        - Horizontally flip the image with probability 1/2
        - Apply random brightness and saturation
    """
    if use_random_flip:
        image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=25.0 / 255.0)
    image = tf.image.random_saturation(image, lower=0.6, upper=1.4)

    # Make sure the image is still in [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label

In [ ]:
def input_fn(is_training, filenames, labels, params):
    """Input function for the dataset.

        Args:
        is_training: (bool) whether to use the train or test pipeline.
                     At training, we shuffle the data and have multiple epochs
        filenames: (list) filenames of the images, as ["data_dir/{label}_IMG_{id}.jpg"...]
        labels: (list) corresponding list of labels
        params: (Params) contains hyperparameters of the model (ex: `params.num_epochs`)
    """
    num_samples = len(filenames)
    assert len(filenames) == len(labels), "Filenames and labels should have same length"

    # Create a Dataset serving batches of images and labels
    # We don't repeat for multiple epochs because we always train and evaluate for one epoch
    parse_fn = lambda f, l: _parse_function(f, l, params.image_size)
    train_fn = lambda f, l: train_preprocess(f, l, params.use_random_flip)

    if is_training:
        dataset = (tf.data.Dataset.from_tensor_slices((tf.constant(filenames), tf.constant(labels)))
            .shuffle(num_samples)  # whole dataset into the buffer ensures good shuffling
            .map(parse_fn, num_parallel_calls=params.num_parallel_calls)
            .map(train_fn, num_parallel_calls=params.num_parallel_calls)
            .batch(params.batch_size)
            .repeat()
            .prefetch(32)  # make sure you always have one batch ready to serve
        )
    else:
        dataset = (tf.data.Dataset.from_tensor_slices((tf.constant(filenames), tf.constant(labels)))
            .map(parse_fn)
            .batch(params.batch_size)
            .repeat()
            .prefetch(32)  # make sure you always have one batch ready to serve
        )
        
    return dataset

create params.json
Notice:i can't upload it, so i write it as a text file.
but in practice you should put it in another folder

In [ ]:
with open("params.json", "w") as text_file:
    text_file.write("{\n"+
    "\"learning_rate\": 1.5e-3,"+
    "\"batch_size\": 64,"+
    "\"num_epochs\": 50,"+
    "\"image_size\": 32,"+
    "\"use_random_flip\": false,"+
    "\"num_labels\": 2,"+
    "\"num_parallel_calls\": 8,"+
    "\"save_summary_steps\": 1"+
    "\n}")
    
paramPath = r'./'
json_path = os.path.join(paramPath , 'params.json')
assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)

read parameter and split data into train and validation(%15)

In [ ]:
params = Params(json_path)
split = int(len(filenames)*0.15)
train_dataset = input_fn(True, filenames[:split], labels[:split], params)
valid_dataset = input_fn(False , filenames[split:], labels[split:], params)

# Plot some sample to ensure we create  correct dataset

The one_shot_iterator method creates an iterator that will be able to iterate once over the dataset. In other words, once we reach the end of the dataset, it will stop yielding elements and raise an Exception.

Now, next_element is a graph’s node that will contain the next element of iterator over the Dataset at each execution (adopted from CS230)

In [ ]:
iterator = train_dataset.make_one_shot_iterator()
next_element = iterator.get_next()
with tf.Session() as sess:
    one_batch = sess.run(next_element)
    print(one_batch[0].shape,' = 64 batch-size & 32x32x3 image')
    for i in range(3):
        plt.figure()
        sample = one_batch[0]
        label = one_batch[1]
        print(label[i])
        plt.imshow(sample[i])
        plt.grid(False)       

# Define Model

Use tf.keras (I think its faster & better than Kera library)

as rule of thumb if you go deeper in ConvNet layer, filters number increase (better to be a power of 2)

In [ ]:
model = Sequential()
        
model.add(Conv2D(3, kernel_size = 3, activation = 'relu', input_shape = (32, 32, 3)))

model.add(Conv2D(filters = 16, kernel_size = 3, activation = 'relu'))
model.add(Conv2D(filters = 16, kernel_size = 3, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(DepthwiseConv2D(kernel_size = 3, strides = 1, padding = 'Same', use_bias = True))
model.add(Conv2D(filters = 32, kernel_size = 1, activation = 'relu'))
model.add(Conv2D(filters = 64, kernel_size = 1, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(DepthwiseConv2D(kernel_size = 3, strides = 2, padding = 'Same', use_bias = True))
model.add(Conv2D(filters = 128, kernel_size = 1, activation = 'relu'))
model.add(Conv2D(filters = 256, kernel_size = 1, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(DepthwiseConv2D(kernel_size = 3, strides = 1, padding = 'Same', use_bias = True))
model.add(Conv2D(filters = 256, kernel_size = 1, activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 512, kernel_size = 1, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(DepthwiseConv2D(kernel_size = 3, strides = 2, padding = 'Same', use_bias = True))
model.add(Conv2D(filters = 512, kernel_size = 1, activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 512, kernel_size = 1, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(DepthwiseConv2D(kernel_size = 3, strides = 1, padding = 'Same', use_bias = True))
model.add(Conv2D(filters = 1024, kernel_size = 1, activation = 'relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 1024, kernel_size = 1, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#model.add(GlobalAveragePooling2D())
model.add(Flatten())

model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation = 'elu'))

model.add(Dense(1, activation = 'sigmoid'))

Most popular optimizer ADAM <3

**base on my [search](https://stats.stackexchange.com/questions/186091/what-loss-function-should-i-use-for-binary-detection-in-face-non-face-detection) and experience log_loss work better than other loss function**

In [ ]:
Adam = tf.keras.optimizers.Adam(lr=params.learning_rate , amsgrad=True)
model.compile(optimizer = Adam, loss = tf.losses.log_loss, metrics = ['accuracy'])
model.summary()

# Defin callbacks

In [ ]:
file_path = 'weights-aerial-cactus.h5'

callbacks = [
        ModelCheckpoint(file_path, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max'),
        ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 3, verbose = 1, mode = 'min', min_lr = 0.00001),
        EarlyStopping(monitor = 'val_loss', min_delta = 1e-10, patience = 15, verbose = 1, restore_best_weights = True)
        ]

if you pass dataset to model.fit() function, you must define **steps_per_epoch** and **validation_steps**

In [ ]:
history = model.fit(train_dataset, validation_data=valid_dataset,
          epochs=50,verbose=True,
          steps_per_epoch=int((len(filenames) - split)/params.batch_size),
          validation_steps=int(split/params.batch_size),
           callbacks = callbacks)

load best weight

In [ ]:
model.load_weights(file_path)

plot loss and acc

In [ ]:
def plot_training_curves(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(acc) + 1)
    
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'g', label='Validation loss')
    plt.title('Losses')
    plt.legend()
    plt.figure()
    
    plt.plot(epochs, acc, 'r', label='Training acc')
    plt.plot(epochs, val_acc, 'g', label='Validation acc')
    plt.title('Accuracies')
    plt.legend()
    plt.figure()
    
    plt.show()

In [ ]:
plot_training_curves(history)

read image from test file and predict result.
at the end save as csv file to submit

**you must put your probability of cactus existence into csv file
so if you replace prob. with  0 and 1 (I mean P less than 0.5 set to zero and upper that set to 1) then ...?
are you get higher score?**

In [ ]:
test_df = pd.read_csv('../input/sample_submission.csv')
X_test = []
images_test = test_df['id'].values

for img_id in tqdm_notebook(images_test):
    X_test.append(cv.imread('../input/test/test/' + img_id))
    
X_test = np.asarray(X_test)
X_test = X_test.astype('float32')
X_test /= 255

y_test_pred = model.predict_proba(X_test)

test_df['has_cactus'] = y_test_pred
test_df.to_csv('aerial-cactus-submission_1.csv', index = False)

for i in range(len(y_test_pred)):
    if y_test_pred[i][0] >= 0.5:
        y_test_pred[i][0] = 1.0
    else:
        y_test_pred[i][0] = 0.0
        
test_df['has_cactus'] = y_test_pred
test_df.to_csv('aerial-cactus-submission_2.csv', index = False)

**The END**